WEB APPLICATION RETE NEURALE

Implementiamo qui una Web Application molto basilare tramite l'utilizzo del modulo Flask di Python.

Per prima cosa installiamo ngrok per Flask che ci permetterà di esporre il server in locale all'esterno. In questo modo potremo raggiungere la nostra Web App senza installare alcun server su Google Drive.

In [1]:
!pip install flask-ngrok

Importiamo tutte le librerie che ci serviranno per l'implementazone della Web App e l'utilizzo della rete neurale

In [2]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for
import numpy as np
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from werkzeug.utils import secure_filename
import os
from PIL import Image

Effettuiamo il montaggio della cartella di Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Otteniamo le componenti della rete neurale che useremo in seguito

In [4]:
model = tensorflow.keras.models.load_model('drive/MyDrive/ProgettoFIA/Modello5generi')
linear_model = tensorflow.keras.models.load_model('drive/MyDrive/ProgettoFIA/modello_linear')

Avviamo la Web App e teniamo traccia della cartella in cui salveremo tutte le immagini che analizzeremo

In [5]:
upload_folder = 'drive/MyDrive/ProgettoFIA/WebApp/images'
app = Flask(__name__,template_folder='drive/MyDrive/ProgettoFIA/WebApp/templates')
run_with_ngrok(app)
app.config["upload_folder"] = upload_folder

Quando accediamo alla pagina "home" tramite una richiesta di tipo "Get", effettuiamo il rendering di "home.html" che contiene semplicemente un form nel quale allegare l'immagine da analizzare, per poi inviarla tramite una richiesta "Post" alla pagina "home" stessa.
Se la richiesta è "Post" prendiamo l'immagine allegata nel form, ne effettuiamo il resize alle dimensioni 100x100 (che è la dimensione che si aspetta di ricevere la nostra rete neurale), ne richiediamo l'analisi da parte della rete neurale ed inviamo il valore di probabilità associato alla categoria più promettente di appartenenza dell'immagine alla pagina maxIndex.

In [6]:
@app.route("/home",methods=["GET","POST"])
def home():
  if request.method == "POST":
    #resize dell'immagine a 100x100
    print(request.files["img"])
    imageFile = request.files["img"]
    imageFile.save(os.path.join(app.config["upload_folder"], secure_filename(imageFile.filename)))
    resizeImage = Image.open(upload_folder+"/"+imageFile.filename)
    resizeImage = resizeImage.resize((100,100), Image.ANTIALIAS)
    resizeImage.save(upload_folder+"/2"+imageFile.filename)
    img_sel = np.array(image.load_img(upload_folder+"/2"+imageFile.filename))
    array = [img_sel]
    array = np.array(array)
    print(array.shape)
    sel = model.predict(array)
    linear_sel = linear_model.predict(sel)
    print(linear_sel)
    result = [i for i, j in enumerate(linear_sel[0]) if j == max(linear_sel[0])][0]
    print(result)
    return redirect(url_for("maxIndex",maxIndex=result))
  else:
    return render_template("home.html")

La pagina maxIndex converte il valore di probabilità associato alla categoria più promettente calcolata dalla rete neurale in una stringa corrispondente al nome di tale categoria. Infine mostra tale nome.

In [7]:
@app.route("/<maxIndex>")
def maxIndex(maxIndex):
  maxIndex = int(maxIndex)
  if maxIndex == 0:
    print("ciao")
    return f"<h1>Libro per bambini</h1>"
  elif maxIndex == 1:
    return f"<h1>Libro di informatica</h1>"
  elif maxIndex == 2:
    return f"<h1>Libro thriller</h1>"
  elif maxIndex == 3:
    return f"<h1>Libro di storia naturale</h1>"
  elif maxIndex == 4:
    return f"<h1>Libro di trasporto</h1>"
  return f"<h1>Immagine non valida</h1>"

Qui avviamo il server contenente la piccola Web App.

In [ ]:
if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6358d08d378a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [10/Feb/2021 08:55:44] "GET / HTTP/1.1" 404 -
[2021-02-10 08:55:44,626] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-1a9f49dd593a>", li

<FileStorage: 'freight-train.jpg' ('image/jpeg')>
(1, 100, 100, 3)


127.0.0.1 - - [10/Feb/2021 08:56:04] "POST /home HTTP/1.1" 302 -
127.0.0.1 - - [10/Feb/2021 08:56:04] "GET /1 HTTP/1.1" 200 -


[[0.00298549 0.9804083  0.00217622 0.0133664  0.0010635 ]]
1


[2021-02-10 08:56:04,505] ERROR in app: Exception on /favicon.ico [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-1a9f49dd593a>", line 3, in maxIndex
    maxIndex = int(maxIndex)
ValueError: i